In [1]:
import pandas as pd
import numpy as np

In [2]:
from glob import glob
from pprint import pprint
import dill
import json

In [3]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
import bokeh.palettes as bpal
from bokeh.models import CategoricalColorMapper
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.models import Legend, LinearAxis, Range1d, DatetimeTickFormatter
from bokeh.io import export_png
output_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

Loading BokehJS ...

In [4]:
import statsmodels.api as sm  
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from pmdarima import auto_arima
from sklearn.preprocessing import StandardScaler

In [5]:
def make_plotgrid(totalrange, ncols):
    """for feeding into bokeh griplot, returns array of rows/columns"""
    nrows = int(np.ceil(totalrange/ncols))
    feeder = iter(range(totalrange))
    plotgrid = [[]]*nrows
    for r in range(nrows):
        thisrow = []
        for c in range(ncols):
            try:
                thisrow.append(next(feeder))
            except:
                thisrow.append(None)
        plotgrid[r] = thisrow
    return plotgrid

In [6]:
def draw_gridplot(indf, columns='first', ncols=3, total_width=900, each_height=300, incolours = ['blue','red','yellow','green','purple','orange']):
    """Grid plots by month in bokeh
    Requires:
    > indf has datetime index
    > if columns are left unspecified (should be a list of col names), then just first column is plotted
    > only takes up to 6 columns, then runs out of colours to plot unless you put more into incolours"""
    if columns == 'first':
        columns = [indf.columns[0]]
    else:
        pass
    indf['Month'] = indf.index.month
    indf['mthname'] = indf.index.month_name()
    
    each_width = int(total_width/ncols)
    
    clrcols = [(col, clr) for col, clr in zip(columns, incolours)]
    dictmonth = {}
    
    for no,mth in enumerate(indf.mthname.unique()):
        dictmonth[no] = figure(plot_width=each_width, plot_height=each_height, title=mth, x_axis_type='datetime')
        for col, clr in clrcols:
            dictmonth[no].line(indf.index[indf.Month == no+1], indf[col][indf.Month == no+1], line_width=1, color = clr)
    
    plotgrid = make_plotgrid(len(indf.mthname.unique()), ncols=ncols)
    for row in range(len(plotgrid)):
        for p in range(len(plotgrid[row])):
            plotgrid[row][p] = dictmonth[plotgrid[row][p]]
    
    the_grid = gridplot(plotgrid)
    show(the_grid)
    return the_grid

In [7]:
feat_files = glob('../data/ready-for-model/*.csv')
feat_files

['../data/ready-for-model/20190808_TAS_interpolated_df_features.csv',
 '../data/ready-for-model/20190808_QLD_interpolated_df_features.csv',
 '../data/ready-for-model/2009-18_NEMtotaldemand.csv',
 '../data/ready-for-model/20190808_VIC_interpolated_df_features.csv',
 '../data/ready-for-model/20190226_SAdf_features.csv',
 '../data/ready-for-model/20190226_TASdf_features.csv',
 '../data/ready-for-model/20190226_NSWdf_features.csv',
 '../data/ready-for-model/20190226_VICdf_features.csv',
 '../data/ready-for-model/20190226_QLDdf_features.csv',
 '../data/ready-for-model/20190808_NSW_interpolated_df_features.csv',
 '../data/ready-for-model/20190808_SA_interpolated_df_features.csv']

In [11]:
fvic = glob('../data/ready-for-model/*VIC*interp*.csv')[0]
fvic

'../data/ready-for-model/20190808_VIC_interpolated_df_features.csv'

In [17]:
ftarget = feat_files = glob('../data/ready-for-model/*NEM*.csv')[0]
ftarget

'../data/ready-for-model/2009-18_NEMtotaldemand.csv'

In [18]:
dftarget = pd.read_csv(ftarget, index_col=0, parse_dates=[0])
dftarget.head(3)

,NSW1,QLD1,SA1,TAS1,VIC1,NEMtotal
SETTLEMENTDATE,,,,,,
2009-01-01 00:30:00,7535.00,5611.54,1310.89,909.71,4799.87,20167.01
2009-01-01 01:00:00,7229.24,5457.34,1272.69,896.63,4646.21,19502.11
2009-01-01 01:30:00,6857.62,5294.12,1178.87,897.52,4950.16,19178.29


### Setting target variable as the electricity demand ('VIC1') up to (& including) year 2018

In [35]:
target = dftarget[['VIC1']][dftarget.index.year != 2019]
target.head(3)

,VIC1
SETTLEMENTDATE,
2009-01-01 00:30:00,4799.87
2009-01-01 01:00:00,4646.21
2009-01-01 01:30:00,4950.16


In [40]:
target.isnull().sum()

VIC1    0
dtype: int64

### dfvic contains all independent variable data, on matching timescale

In [36]:
dfvic = pd.read_csv(fvic, index_col=0, parse_dates=[0])
dfvic.head(3)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayVIC,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,MORWELL_MaxT_85280,MELBOURNE-AIRPORT_MinT_86282,CAPE-NELSON_MinT_90184,MILDURA-AIRPORT_MaxT_76031,MELBOURNE-AIRPORT_MaxT_86282,MORWELL_MinT_85280
SETTLEMENTDATE,,,,,,,,,,,,,,,
2009-01-01 00:30:00,2009-01-01,0.5,2009,0,1,0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:00:00,2009-01-01,1.0,2009,0,1,0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4
2009-01-01 01:30:00,2009-01-01,1.5,2009,0,1,0,0.0,13.8,15.9,20.1,11.2,12.7,27.4,19.9,9.4


In [37]:
dfvic.tail(3)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayVIC,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,MORWELL_MaxT_85280,MELBOURNE-AIRPORT_MinT_86282,CAPE-NELSON_MinT_90184,MILDURA-AIRPORT_MaxT_76031,MELBOURNE-AIRPORT_MaxT_86282,MORWELL_MinT_85280
SETTLEMENTDATE,,,,,,,,,,,,,,,
2018-12-31 23:00:00,2018-12-31,23.0,2018,0,1,0,1.0,21.3,21.4,31.4,15.8,14.9,36.5,25.4,17.1
2018-12-31 23:30:00,2018-12-31,23.5,2018,0,1,0,1.0,21.3,21.4,31.4,15.8,14.9,36.5,25.4,17.1
2019-01-01 00:00:00,2019-01-01,0.0,2019,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
dfvic.isnull().sum()

Date                            0
Hour_of_day                     0
Year                            0
shoulder                        0
summer                          0
winter                          0
workdayVIC                      1
MILDURA-AIRPORT_MinT_76031      1
CAPE-NELSON_MaxT_90184          1
MORWELL_MaxT_85280              1
MELBOURNE-AIRPORT_MinT_86282    1
CAPE-NELSON_MinT_90184          1
MILDURA-AIRPORT_MaxT_76031      1
MELBOURNE-AIRPORT_MaxT_86282    1
MORWELL_MinT_85280              1
dtype: int64

### dfmerged now contains both the target variable and all independent variables, for year 2018

In [23]:
dfmerged = pd.merge(dfvic, target, how='left', left_index=True, right_index=True)
dfmerged.dropna(inplace=True)  # this should only get rid of last row, first half hour of 2019 with some NaN values
dfmerged.head(3)

,Date,Hour_of_day,Year,shoulder,summer,winter,workdayVIC,MILDURA-AIRPORT_MinT_76031,CAPE-NELSON_MaxT_90184,MORWELL_MaxT_85280,MELBOURNE-AIRPORT_MinT_86282,CAPE-NELSON_MinT_90184,MILDURA-AIRPORT_MaxT_76031,MELBOURNE-AIRPORT_MaxT_86282,MORWELL_MinT_85280,VIC1
SETTLEMENTDATE,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,2018-01-01,0.0,2018,0,1,0,0.0,14.3,21.1,26.2,14.0,14.9,31.7,26.2,15.5,4445.07
2018-01-01 00:30:00,2018-01-01,0.5,2018,0,1,0,0.0,14.3,21.1,26.2,14.0,14.9,31.7,26.2,15.5,4251.18
2018-01-01 01:00:00,2018-01-01,1.0,2018,0,1,0,0.0,14.3,21.1,26.2,14.0,14.9,31.7,26.2,15.5,4092.53


In [24]:
target = dfmerged[['VIC1']].copy()
target.isnull().any(axis=1).sum()

0

In [25]:
# daily = seasonal_decompose(target.VIC1, freq = 48)
# fig = plt.figure()  
# fig = daily.plot()  
# fig.set_size_inches(12, 6)
# plt.show()

In [26]:
# weekly = seasonal_decompose(target.d_unexp, freq = 48*7)
# fig = plt.figure()  
# fig = weekly.plot()  
# fig.set_size_inches(12, 6)

In [27]:
target.dropna(inplace=True)
# target.head()

In [28]:
# def stacked(df):
#     df_top = df.cumsum(axis=1)
#     df_bottom = df_top.shift(axis=1).fillna({'y0': 0})[::-1]
#     df_stack = pd.concat([df_bottom, df_top], ignore_index=True)
#     return df_stack

In [29]:
# vfeatures.head(2)

In [30]:
# dfvic[dfvic.index.isin(target.index)].head()

<font color = 'purple'><i>
### This was just for creating graphs for a powerpoint presentation

In [31]:
dtformats = DatetimeTickFormatter(microseconds = ['%fus'],
milliseconds = ['%3Nms', '%S.%3Ns'],
seconds = ['%Ss'],
minsec = [':%M:%S'],
minutes = [':%M', '%Mm'],
hourmin = ['%H:%M'],
hours = ['%Hh', '%H:%M'],
days = ['%d/%m', '%a%d'],
months = ['%m/%Y', '%b %Y'],
years = ['%Y'])

In [32]:
v = figure(plot_height = 400, plot_width = 850, x_axis_type = 'datetime')

v.extra_y_ranges = {'temp' : Range1d(start=0, end=45)}
v.xaxis.formatter = dtformats
v.add_layout(LinearAxis(y_range_name='temp'), 'right')
v.line(target.index, target.VIC1, color = 'purple')
v.line(target.index, dfvic['MELBOURNE-AIRPORT_MaxT_86282'][dfvic.index.isin(target.index)], color = 'orange', y_range_name='temp')
v.line(target.index, dfvic['MELBOURNE-AIRPORT_MinT_86282'][dfvic.index.isin(target.index)], color = 'aqua', y_range_name='temp')
v.vbar(target.index - pd.to_timedelta(15, unit='m'), width=pd.to_timedelta(30, unit='m'), 
       top=dfvic['workdayVIC'][dfvic.index.isin(target.index)]*100, fill_alpha=0.3, fill_color = 'green', 
       line_alpha=0, y_range_name='temp')

# show(v)

GlyphRenderer(id='1058', ...)

In [33]:
# z = figure(plot_height = 400, plot_width = 850, x_axis_type = 'datetime')

# srange = pd.to_datetime('20-4-2018', dayfirst=True)
# erange = pd.to_datetime('24-4-2018', dayfirst=True)

# myfontsize = '14pt'
# labelfont = '18pt'

# z.x_range.start = srange
# z.x_range.end   = erange
# z.y_range.start = 2000
# z.y_range.end   = 6500
# z.yaxis.major_label_text_font_size = myfontsize
# z.xaxis.major_label_text_font_size = myfontsize
# z.xaxis.formatter = dtformats

# z.yaxis.axis_label = 'Electricity Demand (MW)'
# z.yaxis.axis_label_text_font_size = labelfont
# z.yaxis.axis_label_standoff = 20


# z.extra_y_ranges = {'temp' : Range1d(start=0, end=45)}
# z.add_layout(LinearAxis(y_range_name='temp', major_label_text_font_size=myfontsize, 
#                         axis_label='Temperature (Min/Max) - deg C', axis_label_text_font_size=labelfont, axis_label_standoff = 20)
#              , 'right')

# # Elec Demand
# z.line(target.index, target.VIC1, color = 'purple', line_width=3)

# # # # Melb max temp
# # z.line(target.index, dfvic['MELBOURNE-AIRPORT_MaxT_86282'][dfvic.index.isin(target.index)], color = 'orange', 
# #        y_range_name='temp', line_width=1.5, legend='Daily Max Temp (Melb Airport)')
# # # # Melb min temp
# # z.line(target.index, dfvic['MELBOURNE-AIRPORT_MinT_86282'][dfvic.index.isin(target.index)], color = 'aqua', 
# #        y_range_name='temp', line_width=1.5, legend='Daily Min Temp (Melb Airport)')

# # is workday
# workx = target.index + pd.to_timedelta(15, unit='m')
# worky = dfvic['workdayVIC'][dfvic.index.isin(target.index)]*100
# z.vbar(workx, width=pd.to_timedelta(30, unit='m'), top=worky, fill_alpha=0.3, fill_color = 'green', line_alpha=0, y_range_name='temp', legend='Is a workday')

# z.legend.location = 'top_center'
# z.legend.label_text_font_size = '12pt'

# show(z)

In [34]:
# target['Time'] = target.index.time
# target.head()

In [ ]:
# yravg = pd.pivot_table(data=target[['VIC1','Time']], index='Time')

In [ ]:
# y = figure(plot_height = 400, plot_width = 600, x_axis_type = 'datetime')

# y.x_range.start=pd.to_datetime('1-1-1970 00:00:00')
# y.x_range.end  =pd.to_datetime('1-1-1970 23:30:00')

# y.yaxis.major_label_text_font_size = myfontsize
# y.xaxis.major_label_text_font_size = myfontsize

# y.yaxis.axis_label = 'Electricity Demand (MW)'
# y.yaxis.axis_label_text_font_size = labelfont
# y.yaxis.axis_label_standoff = 20

# y.xaxis.formatter = DatetimeTickFormatter(microseconds = ['%fus'],
# milliseconds = ['%3Nms', '%S.%3Ns'],
# seconds = ['%Ss'],
# minsec = [':%M:%S'],
# minutes = [':%M', '%Mm'],
# hourmin = ['%H:%M'],
# hours = ['%H:00', '%H:%M'],
# days = ['%H:00'], #['%d/%m', '%a%d'],
# months = ['%m/%Y', '%b %Y'],
# years = ['%Y'])

# y.line(yravg.index, yravg.VIC1, color='purple', line_width=2.5)

# show(y)

In [ ]:
# draw_gridplot(target, columns=['wseason'])

In [ ]:
newtarget = dfmerged[['VIC1']].copy()

In [ ]:
# nweekly = seasonal_decompose(target.VIC1, freq = 48*7)
# fig = plt.figure()  
# fig = nweekly.plot()  
# fig.set_size_inches(12, 6)
# plt.show()

In [ ]:
# newtarget['wtrend'] = nweekly.trend
# newtarget['wseason'] = nweekly.seasonal
# newtarget['wresid'] = nweekly.resid
# newtarget.dropna(inplace=True)
# newtarget.head()

In [ ]:
# draw_gridplot(newtarget, columns=['wseason'])

In [ ]:
# ndaily = seasonal_decompose(target.wtrend, freq = 48)
# fig = plt.figure()  
# fig = ndaily.plot()  
# fig.set_size_inches(12, 6)
# plt.show()

In [ ]:
# len(newtarget) / 4

In [ ]:
newtarget.head()

In [ ]:
# newtarget['w_unexp'] = newtarget[['wtrend','wresid']].sum(axis=1)
# newtarget.head(2)

In [ ]:
# sumwin = seasonal_decompose(newtarget.wtrend, freq=int(len(newtarget) / 4))
# fig = plt.figure()  
# fig = sumwin.plot()  
# fig.set_size_inches(12, 6)
# plt.show()

In [ ]:
# newtarget['strend'] = sumwin.trend
# newtarget['sseason'] = sumwin.seasonal
# newtarget['sresid'] = sumwin.resid
# newtarget.dropna(inplace=True)
# newtarget.head()

In [ ]:
# newtarget['unexplained'] = newtarget[['sseason','sresid','wresid']].sum(axis=1)
# newtarget['Total'] = newtarget[['strend','wseason','unexplained']].sum(axis=1)
# newtarget.head(3)

In [ ]:
# newtarget[abs(newtarget.VIC1 - newtarget.Total) > 0.01]

In [ ]:
# fig, ax = plt.subplots(figsize=(12,4))
# ax.plot(newtarget.index, newtarget.unexplained)
# plt.show()

<font color = 'purple'><br>
### Creating StandardScaled set of exogenous features

In [ ]:
vfeatures = dfvic[dfvic.index.isin(newtarget.index)].drop(columns=['Date','Year','Hour_of_day'])
vfeatures.head()

In [ ]:
ssv = StandardScaler()

In [ ]:
vfeatures = pd.DataFrame(data=ssv.fit_transform(vfeatures), index=vfeatures.index, columns=vfeatures.columns)
vfeatures.head()

In [ ]:
# both_for_plot = pd.merge(vfeatures, newtarget[['unexplained']], how='inner', left_index=True, right_index=True)

In [ ]:
def multi_scatter(indf, ycol, xcols_list, no_cols=4):

    no_rows = int(np.ceil(len(xcols_list)/no_cols))
    plotlen = int(round((15/no_cols)*no_rows))
    sfig, saxes = plt.subplots(nrows=no_rows, ncols=no_cols, figsize = (15,plotlen), sharey=True)

    saxes = saxes.flatten()

    for ct, a in enumerate(xcols_list):
        saxes[ct].scatter(x=indf[a], y= indf[ycol])
        saxes[ct].set(title=f"{ycol} vs {a}")

    sfig.subplots_adjust(wspace= 0.2, hspace=0.35)
#     plt.savefig("correlation-plots.png")
    plt.show()

In [ ]:
# multi_scatter(both_for_plot, ycol='unexplained', xcols_list=vfeatures.columns)

In [ ]:
# newtarget['vdif'] = newtarget.VIC1.diff()
# newtarget.dropna(inplace=True)
# newtarget.head(3)

In [ ]:
# dif = figure(plot_height = 400, plot_width = 850, x_axis_type = 'datetime')

# dif.line(newtarget.index, newtarget.VIC1, color = 'purple')
# dif.line(newtarget.index, newtarget.vdif, color = 'green')
# dif.xaxis.formatter = dtformats

# show(dif)

In [ ]:
# fig, ax = plt.subplots(figsize=(15,5))
# plot_acf(newtarget.vdif.values, lags=100, ax=ax)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(15,5))
# plot_pacf(newtarget.vdif.values, lags=100, ax=ax)
# plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12, center=False).mean()
    rolstd = timeseries.rolling(window=12, center=False).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in list(dftest[4].items()):
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput) 

In [ ]:
# test_stationarity(newtarget.unexplained)

In [ ]:
# newtarget
# print(len(newtarget))
# print(len(newtarget[vfeatures.iloc[1:].index == newtarget.index]))

In [ ]:
# print(newtarget.index.min())
# print(newtarget.index.max())

In [ ]:
%%time
# Variables
endog = newtarget.VIC1
exog = vfeatures.iloc[1:]

# Fit the model
mod = sm.tsa.statespace.SARIMAX(endog, exog, order=(1,1,1), seasonal_order=(1,0,0,48))
res = mod.fit(disp=False)
print(res.summary())

In [ ]:
# with open('sarimax111x100s48.pickle', 'wb') as outfile:
#     dill.dump(res, outfile)
#     ## this doesn't work, it's 2.6GB....

In [ ]:
%%time
yhat = res.predict(start=newtarget.index.min(), end=newtarget.index.max())

In [ ]:
newtarget.head()

In [ ]:
newtarget['pred'] = yhat
newtarget.head(3)

In [ ]:
newtarget.index.max()

In [ ]:
forecast = res.get_prediction(start=newtarget.index.min(), end=newtarget.index.max(), dynamic = True)

In [ ]:
forecast.prediction_results.results

In [ ]:
# newtarget['forecast'] = forecast.prediction_results()

In [ ]:
# [r for r in forecast.prediction_results.results]

In [ ]:
newtarget

In [ ]:
p = figure(x_axis_type='datetime', plot_width = 850, plot_height = 400)

p.line(newtarget.index, newtarget.VIC1, color='blue')
p.line(newtarget.index, newtarget.pred, color='pink')
# p.line(newtarget.index, newtarget.forecast, color='green')

show(p)